# Analysis of Excess Deaths & Vaccine Status by US States

In [30]:
import pandas as pd
import numpy as np
import datetime
import calendar
import requests
import os

import statsmodels.formula.api as smf
import statsmodels.api as sm

In [75]:
start_date = datetime.date(2022, 9, 1)

In [76]:
end_date = datetime.date(2023, 1, 31)

### Data Sources

In [77]:
# https://data.cdc.gov/NCHS/Excess-Deaths-Associated-with-COVID-19/xkkf-xrst
excess_deaths = "Excess_Deaths_Associated_with_COVID-19.csv"
vaccinated = "https://github.com/owid/covid-19-data/raw/master/public/data/vaccinations/us_state_vaccinations.csv"
state_demographics = "https://corgis-edu.github.io/corgis/datasets/csv/state_demographics/state_demographics.csv"

In [78]:
ef = pd.read_csv(excess_deaths)

In [79]:
ef['Excess Estimate'] = pd.to_numeric(ef['Excess Estimate'], errors='coerce').fillna(0)

In [80]:
ef['Week Ending Date'] = pd.to_datetime(ef['Week Ending Date'], errors='coerce').dt.date

In [81]:
date_mask = (ef['Week Ending Date'] >= start_date) & (ef['Week Ending Date'] <= end_date)
cause_mask = (ef['Outcome'] == 'All causes, excluding COVID-19')
sum(date_mask), sum(cause_mask)

(2916, 16902)

In [82]:
ef = ef[date_mask & cause_mask]

In [83]:
ef = ef.groupby('State').sum()

In [84]:
ef = ef['Excess Estimate'].to_frame()

### State Demographics

In [85]:
df = pd.read_csv(state_demographics)

In [86]:
df = df.set_index('State')

In [87]:
df = df.rename(columns={'Population.2014 Population':'Population', 'Age.Percent 65 and Older':'Pop65', 
                        'Income.Per Capita Income':'Income', 'Income.Persons Below Poverty Level':'Poor',
                       'Population.Population per Square Mile':'PopDensity'})

In [88]:
df = df['Population'].to_frame()

## Vaccinated

In [89]:
vf = pd.read_csv(vaccinated)

In [90]:
vf = vf.groupby('location').max()
vf = vf['total_boosters_per_hundred'].to_frame()

In [91]:
vf.columns = ['Boosted']

## Merge

In [92]:
df = df.merge(vf, how='left', right_index=True, left_index=True)

In [93]:
df = df.merge(ef, how='left', right_index=True, left_index=True)

In [94]:
df['ExcessPer100k'] = df['Excess Estimate'] * 100000 / df['Population']

In [95]:
df['Constant'] = 1

## Model with Intercept

In [99]:
mask = df['ExcessPer100k'] > 0
model = sm.OLS(df[mask]['ExcessPer100k'], df[mask][['Boosted', 'Constant']], missing='drop').fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          ExcessPer100k   R-squared:                       0.094
Model:                            OLS   Adj. R-squared:                  0.075
Method:                 Least Squares   F-statistic:                     4.993
Date:                Tue, 17 Jan 2023   Prob (F-statistic):             0.0301
Time:                        15:06:42   Log-Likelihood:                -190.06
No. Observations:                  50   AIC:                             384.1
Df Residuals:                      48   BIC:                             387.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Boosted        0.2134      0.096      2.234      0.030       0.021       0.405
Constant       6.3568      5.556      1.144      0.258      -4.814      17.528
==============================================================================
Omnibus:                       15.023   Durbin-Watson:                   2.143
Prob(Omnibus):                  0.001   Jarque-Bera (JB):               17.541
Skew:                           1.140   Prob(JB):                     0.000155
Kurtosis:                       4.794   Cond. No.                         207.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Model with no Intercept

In [100]:
mask = df['ExcessPer100k'] > 0
model = sm.OLS(df[mask]['ExcessPer100k'], df[mask][['Boosted']], missing='drop').fit()
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 OLS Regression Results                                
=======================================================================================
Dep. Variable:          ExcessPer100k   R-squared (uncentered):                   0.740
Model:                            OLS   Adj. R-squared (uncentered):              0.735
Method:                 Least Squares   F-statistic:                              139.4
Date:                Tue, 17 Jan 2023   Prob (F-statistic):                    6.11e-16
Time:                        15:06:44   Log-Likelihood:                         -190.73
No. Observations:                  50   AIC:                                      383.5
Df Residuals:                      49   BIC:                                      385.4
Df Model:                           1                                                  
Covariance Type:            nonrobust                                                  
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Boosted        0.3183      0.027     11.808      0.000       0.264       0.372
==============================================================================
Omnibus:                       10.612   Durbin-Watson:                   2.246
Prob(Omnibus):                  0.005   Jarque-Bera (JB):               10.642
Skew:                           0.900   Prob(JB):                      0.00489
Kurtosis:                       4.367   Cond. No.                         1.00
==============================================================================

Notes:
[1] R² is computed without centering (uncentered) since the model does not contain a constant.
[2] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""